### Add time for the future prediction
- Doesn't work. The time injection needs to be position embeding, like Transformer. I guess
![title](images/multiple_state_plus_time.jpg)

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib notebook
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
w,h=10,10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Utils

In [3]:
def delete_index(T, index=None):
    if index is None:
        return T
    elif index is -1:
        return T[:index]
    else:
        return torch.cat([T[:index], T[index+1:]])
delete_index(torch.arange(30).reshape(3,10), 1), \
delete_index(torch.arange(30).reshape(3,10)), \
delete_index(torch.arange(30).reshape(3,10), -1)

(tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]))

In [4]:
def delete_indexes(T, indexes=None):
    if indexes is None:
        return T
    else:
        if indexes[0] > indexes[1]: # ex indexes = [0, -1]
            return T
        elif indexes[1] == -1: # ex indexes = [-3, -1]
            return T[:indexes[0]]
        return torch.cat([T[:indexes[0]], T[indexes[1]+1:]])
arr = torch.arange(15).reshape(5,3)
arr, delete_indexes(arr, [0,2])

(tensor([[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11],
         [12, 13, 14]]),
 tensor([[ 9, 10, 11],
         [12, 13, 14]]))

In [5]:
def fifo(current_arr, arr):
    current_arr = torch.unsqueeze(current_arr, 0)
    arr = delete_index(arr, index=0)
    return torch.cat((arr, current_arr))
fifo(torch.arange(10), torch.zeros(30).reshape(3,10))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [6]:
def push(arr, current_arr):
    if arr is None:
        return torch.unsqueeze(current_arr, 0)
    current_arr = torch.unsqueeze(current_arr, 0)
    return torch.cat((arr, current_arr))
push(torch.zeros(10).reshape(1,10), torch.arange(10)), push(None, torch.arange(10))

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]),
 tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]))

In [7]:
def jump_bound(position):
    if position[0] >= w:
        position[0] = position[0] - w
    elif position[0] < 0:
        position[0] = w + position[0]
    if position[1] >= h:
        position[1] = position[1] - w
    elif position[1] < 0:
        position[1] = h + position[1]
    return position
jump_bound([10, -1])

[0, 9]

### Env

In [8]:
class MomentumObject:
    def __init__(self, w, h, device):
        self.w, self.h = w,h
        self._position = torch.zeros((2), dtype=torch.int).to(device)
        self._speed =  torch.zeros((2), dtype=torch.int).to(device)
    
    @property
    def position(self):
        return torch.clone(self._position)
    @property
    def speed(self):
        return torch.clone(self._speed)
    @speed.setter
    def speed(self, speed):
        self._speed = speed
    def step(self):
        self._position += self._speed
        jump_bound(self._position)
    def render(self, predict_position=None):
        state = torch.zeros((w,h))
        state[self._position[0], self._position[1]] = 1
        if predict_position is not None:
            predict_position = predict_position.type(torch.int)
            state[predict_position[0], predict_position[1]] = 0.2
        return state
momentum_object = MomentumObject(w, h, device)
momentum_object.step()
momentum_object.render()

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

### Add to Buffer

In [9]:
class MemoryReplay:
    def __init__(self, num_step_per_action, num_action_buffer, num_state, num_action, env):
        self.num_step_per_action = num_step_per_action
        self.num_action_buffer = num_action_buffer
        self.env = env
        self._states_actions = torch.zeros((num_step_per_action*num_action_buffer, num_state+num_action), dtype=torch.double).to(device)
        self.current_step = -1
        self._num_buffer = self.num_step_per_action * self.num_action_buffer
    def step(self):
        self.current_step += 1
        if self.current_step >= num_step_per_action:
            self.current_step = 0
        self.env.step()
        if self.current_step % self.num_step_per_action == 0:
            self.env.speed = torch.randint(-2, 2, size=(2,), device=device)
        state_action = torch.cat((self.env.position, self.env.speed))
        self._states_actions = fifo(state_action, self._states_actions)
    @property
    def states_actions(self):
        return torch.clone(self._states_actions)
    def warmup(self):
        for _ in range(self._num_buffer):
            self.step()
num_step_per_action = 4
num_action_buffer = 3
memory = MemoryReplay(num_step_per_action=num_step_per_action, 
                      num_action_buffer=num_action_buffer, 
                      num_state=2, 
                      num_action=2, 
                      env=momentum_object)
memory.warmup()

In [10]:
memory.step()
memory.step()
memory.states_actions, memory.current_step

(tensor([[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0., -1.],
         [ 0.,  9.,  0., -1.],
         [ 0.,  8.,  0., -1.],
         [ 0.,  7.,  0., -1.],
         [ 0.,  6.,  1.,  1.],
         [ 1.,  7.,  1.,  1.],
         [ 2.,  8.,  1.,  1.],
         [ 3.,  9.,  1.,  1.],
         [ 4.,  0., -2.,  1.],
         [ 2.,  1., -2.,  1.]], device='cuda:0', dtype=torch.float64),
 1)

##### Map direct indexes to repeated indexes

In [11]:
memory.current_step

1

In [12]:
def map_repeat_indexes(action_group_id):
    shift_start = (memory.current_step)%num_step_per_action
    if action_group_id == 0:
        x = torch.arange(num_step_per_action - shift_start)
    elif action_group_id == num_action_buffer:
        x = torch.arange(shift_start)
    else:
        x = torch.arange(num_step_per_action)
    repeats = torch.flip(x, dims=(0,)) + 1
    current_indexes = torch.repeat_interleave(x, repeats=repeats, dim=0)
    current_indexes += (action_group_id*num_step_per_action)
    
    if action_group_id == 0:
        current_indexes += shift_start
    return current_indexes - shift_start
def map_delta_time(action_group_id):
    shift_start = memory.current_step%num_step_per_action
    count = num_step_per_action
    if action_group_id == 0:
        count -= shift_start
    elif action_group_id == num_action_buffer:
        count -= (num_step_per_action-shift_start)
    return torch.cat([torch.arange(i) for i in range(count, 0, -1)])

def get_repeat_state(state):
    # map feature
    current_indexes = [map_repeat_indexes(i) for _, i in enumerate(range(num_action_buffer + 1))]
    current_indexes =  torch.cat(current_indexes).to(device)
    current_indexes = torch.index_select(state, dim=0, index=current_indexes)
    
    # map delta time
    num_actions = num_action_buffer + 1 if memory.current_step != 0 else num_action_buffer
    delta_time = [map_delta_time(i) for _, i in enumerate(range(num_actions))]
    delta_time = torch.cat(delta_time).to(device)
    delta_time = torch.unsqueeze(delta_time, 1)

    return torch.cat((current_indexes, delta_time), dim=1)
get_repeat_state(memory.states_actions[:,:2])

tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 2.],
        [0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 9., 0.],
        [0., 9., 1.],
        [0., 9., 2.],
        [0., 9., 3.],
        [0., 8., 0.],
        [0., 8., 1.],
        [0., 8., 2.],
        [0., 7., 0.],
        [0., 7., 1.],
        [0., 6., 0.],
        [1., 7., 0.],
        [1., 7., 1.],
        [1., 7., 2.],
        [1., 7., 3.],
        [2., 8., 0.],
        [2., 8., 1.],
        [2., 8., 2.],
        [3., 9., 0.],
        [3., 9., 1.],
        [4., 0., 0.],
        [2., 1., 0.]], device='cuda:0', dtype=torch.float64)

In [13]:
def map_target_indexes():
    acc_count = 0
    def map_target_index(action_group_id):
        nonlocal acc_count
        shift_start = memory.current_step%num_step_per_action
        count = num_step_per_action
        if action_group_id == 0:
            count -= shift_start
        elif action_group_id == num_action_buffer:
            count -= (num_step_per_action-shift_start)
        acc_count += count
        return torch.cat([torch.arange(_range) + index for index,_range in enumerate(range(count, 0, -1))])
    num_actions = num_action_buffer + 1 if memory.current_step != 0 else num_action_buffer
    indexes = [acc_count + map_target_index(i) for i in range(num_actions)]
    return torch.cat(indexes)

def get_target_states(states):
    indexes = map_target_indexes().to(device)
    return torch.index_select(states, dim=0, index=indexes)

torch.cat((get_target_states(memory.states_actions[:,:2]), get_repeat_state(memory.states_actions[:,:2])), axis=1)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 9., 0., 9., 0.],
        [0., 8., 0., 9., 1.],
        [0., 7., 0., 9., 2.],
        [0., 6., 0., 9., 3.],
        [0., 8., 0., 8., 0.],
        [0., 7., 0., 8., 1.],
        [0., 6., 0., 8., 2.],
        [0., 7., 0., 7., 0.],
        [0., 6., 0., 7., 1.],
        [0., 6., 0., 6., 0.],
        [1., 7., 1., 7., 0.],
        [2., 8., 1., 7., 1.],
        [3., 9., 1., 7., 2.],
        [4., 0., 1., 7., 3.],
        [2., 8., 2., 8., 0.],
        [3., 9., 2., 8., 1.],
        [4., 0., 2., 8., 2.],
        [3., 9., 3., 9., 0.],
        [4., 0., 3., 9., 1.],
        [4., 0., 4., 0., 0.],
        [2., 1., 2., 1., 0.]], device='cuda:0', dtype=torch.float64)

### SD + Normalize function

In [14]:
def standardize_state(state):
    ''' 
    state: [0, 9], [0, 9], action1, action2
    return: [-4.5, 4.5], [-4.5, 4.5], action1, action2
    '''
    state = torch.clone(state)
    state[:, 0] = state[:, 0] - ((w-1)/2)
    state[:, 1] = state[:, 1] - ((h-1)/2)
    return state
def normalize_state(state):
    ''' 
    state: [0, 9], [0, 9]
    return [0, 1], [0, 1] 
    '''
    state = torch.clone(state)
    state[:, 0] = (state[:, 0] + 0.5) / w
    state[:, 1] = (state[:, 1] + 0.5) / h
    return state
def denormalize_state(state):
    ''' 
    state: [0, 1], [0, 1] 
    return [0, 9], [0, 9]
    '''
    state = torch.clone(state)
    state[:, 0] = state[:, 0]*w -0.5
    state[:, 1] = state[:, 1]*h -0.5
    return state
a = denormalize_state(torch.tensor([[0.,1.]]))
a, torch.round(a)
# normalize_state(torch.tensor([[0.,9.]]))

(tensor([[-0.5000,  9.5000]]), tensor([[-0., 10.]]))

### Train

In [16]:
def train(i):
    memory.step()
    encoder.train()
    optimizer.zero_grad()
    current_state = standardize_state(memory.states_actions)
    predict_state = encoder(current_state)
    
    state_prediction_feature = get_repeat_state(predict_state)
    state_prediction = decoder(state_prediction_feature)
    
    future_label = normalize_state(memory.states_actions[:,:2])
    future_label = get_target_states(future_label)
    loss = mse_loss(state_prediction, future_label.detach())
    loss.mean().backward(retain_graph=True)
    
#     state_prediction = torch.round(denormalize_state(state_prediction))
#     miss_distance = (future_label - state_prediction).float().sum().cpu().detach().numpy()
    print("loss[", i,"]:", loss.mean().cpu().detach().numpy()  )#, 'miss_distance', miss_distance)
    
    loss.mean().backward()
    optimizer.step()
    
torch.manual_seed(1)
encoder = nn.Sequential(
                nn.Linear(4,16),
                nn.ReLU(),
                nn.Linear(16,32),
                nn.ReLU(),
                nn.Linear(32,8),
                nn.ReLU(),
                nn.Linear(8,8),
                nn.Sigmoid()).to(device).to(torch.float64)
decoder = nn.Sequential(
                nn.Linear(9,16),
                nn.ReLU(),
                nn.Linear(16,32),
                nn.ReLU(),
                nn.Linear(32,8),
                nn.ReLU(),
                nn.Linear(8,2),
                nn.Sigmoid()).to(device).to(torch.float64)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)
mse_loss = nn.MSELoss(reduction='none')

lowest_loss_index = None
for i in range(5000):
    lowest_loss_index = train(i)

loss[ 0 ]: 0.08593356949219405
loss[ 1 ]: 0.09462426933906284
loss[ 2 ]: 0.10224974290196935
loss[ 3 ]: 0.10934236189476217
loss[ 4 ]: 0.12094394519248865
loss[ 5 ]: 0.12833444762326615
loss[ 6 ]: 0.12775238415578963
loss[ 7 ]: 0.11728585534620663
loss[ 8 ]: 0.10140497366133239
loss[ 9 ]: 0.09329827234341748
loss[ 10 ]: 0.08741403567815752
loss[ 11 ]: 0.08532729623792266
loss[ 12 ]: 0.08481539786706316
loss[ 13 ]: 0.07982827041745372
loss[ 14 ]: 0.07522131305862735
loss[ 15 ]: 0.07191410148315648
loss[ 16 ]: 0.0725968664145849
loss[ 17 ]: 0.0794818660750144
loss[ 18 ]: 0.08775361075230197
loss[ 19 ]: 0.08642662471917151
loss[ 20 ]: 0.07729482849333269
loss[ 21 ]: 0.07261071658313409
loss[ 22 ]: 0.063961111330741
loss[ 23 ]: 0.05798444927695617
loss[ 24 ]: 0.05972668327285689
loss[ 25 ]: 0.05492248012310364
loss[ 26 ]: 0.04886502346276846
loss[ 27 ]: 0.047376741250985975
loss[ 28 ]: 0.05951679393595343
loss[ 29 ]: 0.06658280130877661
loss[ 30 ]: 0.0772226421137635
loss[ 31 ]: 0.08429997

loss[ 267 ]: 0.08346798020524437
loss[ 268 ]: 0.08419400467484132
loss[ 269 ]: 0.08133581668510258
loss[ 270 ]: 0.07451289223736297
loss[ 271 ]: 0.06492411240757659
loss[ 272 ]: 0.059290264722621985
loss[ 273 ]: 0.05741741980490507
loss[ 274 ]: 0.05404673210775177
loss[ 275 ]: 0.053678574452572354
loss[ 276 ]: 0.05804286800450413
loss[ 277 ]: 0.0544756600363533
loss[ 278 ]: 0.053385799899288655
loss[ 279 ]: 0.05278884894610427
loss[ 280 ]: 0.054259651390410994
loss[ 281 ]: 0.05596783763519371
loss[ 282 ]: 0.05681318182183028
loss[ 283 ]: 0.059325222617057916
loss[ 284 ]: 0.07153562109836957
loss[ 285 ]: 0.08045562698293574
loss[ 286 ]: 0.08911169766171083
loss[ 287 ]: 0.09407613069990081
loss[ 288 ]: 0.09476924308454537
loss[ 289 ]: 0.08946490455836362
loss[ 290 ]: 0.08298370973701956
loss[ 291 ]: 0.07433821476399377
loss[ 292 ]: 0.06914224212037962
loss[ 293 ]: 0.07294176588432438
loss[ 294 ]: 0.07371964770338406
loss[ 295 ]: 0.06603894763871684
loss[ 296 ]: 0.05525246295619112
loss[ 

loss[ 516 ]: 0.038105984226542176
loss[ 517 ]: 0.03854184674308288
loss[ 518 ]: 0.03867954724189822
loss[ 519 ]: 0.05499050746658641
loss[ 520 ]: 0.06902173847060306
loss[ 521 ]: 0.07352055104097387
loss[ 522 ]: 0.07876075533662576
loss[ 523 ]: 0.08038816641034133
loss[ 524 ]: 0.08637395764258986
loss[ 525 ]: 0.09209826177783677
loss[ 526 ]: 0.09843510278754763
loss[ 527 ]: 0.11932765037890465
loss[ 528 ]: 0.1241598677861691
loss[ 529 ]: 0.12048763521834445
loss[ 530 ]: 0.10562277216332346
loss[ 531 ]: 0.09345642219224247
loss[ 532 ]: 0.08882703825501101
loss[ 533 ]: 0.08276785480528537
loss[ 534 ]: 0.08100754219343438
loss[ 535 ]: 0.07569216647008468
loss[ 536 ]: 0.07015976208427395
loss[ 537 ]: 0.05952889514164052
loss[ 538 ]: 0.04655347558607799
loss[ 539 ]: 0.03991291544885202
loss[ 540 ]: 0.03676498638026889
loss[ 541 ]: 0.03571758639994158
loss[ 542 ]: 0.04118543983298675
loss[ 543 ]: 0.04133214509751491
loss[ 544 ]: 0.03684002541428893
loss[ 545 ]: 0.0357263737974845
loss[ 546 ]

loss[ 785 ]: 0.03572772724333462
loss[ 786 ]: 0.03442650111521694
loss[ 787 ]: 0.05445462755325874
loss[ 788 ]: 0.06810646894200083
loss[ 789 ]: 0.07620159864477687
loss[ 790 ]: 0.08228341286543384
loss[ 791 ]: 0.08314940517898446
loss[ 792 ]: 0.07942314490121793
loss[ 793 ]: 0.07987970427766575
loss[ 794 ]: 0.07548311162303238
loss[ 795 ]: 0.07474631120717853
loss[ 796 ]: 0.07909144365587431
loss[ 797 ]: 0.0702183331590558
loss[ 798 ]: 0.05569401481949147
loss[ 799 ]: 0.05381606458730955
loss[ 800 ]: 0.05421459398051424
loss[ 801 ]: 0.05062371957556707
loss[ 802 ]: 0.056212065783963955
loss[ 803 ]: 0.056526890844650304
loss[ 804 ]: 0.05135594268505847
loss[ 805 ]: 0.049961105901055805
loss[ 806 ]: 0.052653700110720715
loss[ 807 ]: 0.049377237853221165
loss[ 808 ]: 0.044796701799054955
loss[ 809 ]: 0.04271599749382925
loss[ 810 ]: 0.04463733538847576
loss[ 811 ]: 0.05393556478087747
loss[ 812 ]: 0.058314684820262044
loss[ 813 ]: 0.04970091906918543
loss[ 814 ]: 0.05340869363754777
loss

loss[ 1048 ]: 0.041564258217997335
loss[ 1049 ]: 0.03927251281874649
loss[ 1050 ]: 0.04017299164980295
loss[ 1051 ]: 0.04860108965000517
loss[ 1052 ]: 0.05049600774306538
loss[ 1053 ]: 0.046522185938390255
loss[ 1054 ]: 0.038201486860217695
loss[ 1055 ]: 0.032613973867502824
loss[ 1056 ]: 0.032414256647468165
loss[ 1057 ]: 0.034282513028113776
loss[ 1058 ]: 0.035110244020788564
loss[ 1059 ]: 0.03399324587911262
loss[ 1060 ]: 0.03249797673760573
loss[ 1061 ]: 0.026906029436782053
loss[ 1062 ]: 0.02117357042021094
loss[ 1063 ]: 0.02537112938888487
loss[ 1064 ]: 0.03891624377025078
loss[ 1065 ]: 0.03955226558901917
loss[ 1066 ]: 0.04039087898118865
loss[ 1067 ]: 0.043925223809002927
loss[ 1068 ]: 0.054963177817848416
loss[ 1069 ]: 0.06001177451549601
loss[ 1070 ]: 0.06075810641535358
loss[ 1071 ]: 0.05683992624828137
loss[ 1072 ]: 0.04937466236862771
loss[ 1073 ]: 0.042975036460973316
loss[ 1074 ]: 0.035893500568064576
loss[ 1075 ]: 0.03066466499173161
loss[ 1076 ]: 0.04117892784276745
lo

...

loss[ 4010 ]: 0.05257841425715123
loss[ 4011 ]: 0.06255093557887056
loss[ 4012 ]: 0.06477948377010215
loss[ 4013 ]: 0.060367654004322945
loss[ 4014 ]: 0.04871604852193466
loss[ 4015 ]: 0.04719723897751519
loss[ 4016 ]: 0.046034383176385474
loss[ 4017 ]: 0.040285738591214315
loss[ 4018 ]: 0.03148535678761866
loss[ 4019 ]: 0.041700293907572045
loss[ 4020 ]: 0.04380012400715586
loss[ 4021 ]: 0.03929962620672666
loss[ 4022 ]: 0.0304486173193092
loss[ 4023 ]: 0.028802420819760248
loss[ 4024 ]: 0.033600940228388186
loss[ 4025 ]: 0.03499068713698356
loss[ 4026 ]: 0.03737131317614483
loss[ 4027 ]: 0.045802700151642825
loss[ 4028 ]: 0.058449492877860616
loss[ 4029 ]: 0.054919298734027865
loss[ 4030 ]: 0.04664172334801882
loss[ 4031 ]: 0.0461437986009688
loss[ 4032 ]: 0.04894731456460253
loss[ 4033 ]: 0.04859403987123253
loss[ 4034 ]: 0.04836366733822683
loss[ 4035 ]: 0.053549825487352265
loss[ 4036 ]: 0.050792527682032095
loss[ 4037 ]: 0.042493271489763965
loss[ 4038 ]: 0.04317992630810012
loss

loss[ 4261 ]: 0.05083865618327793
loss[ 4262 ]: 0.038712310176451326
loss[ 4263 ]: 0.037670233592609306
loss[ 4264 ]: 0.03563430702125284
loss[ 4265 ]: 0.026452247101703685
loss[ 4266 ]: 0.01288144067543218
loss[ 4267 ]: 0.012321421467319104
loss[ 4268 ]: 0.012199368474165638
loss[ 4269 ]: 0.00866796956815027
loss[ 4270 ]: 0.006892151342719569
loss[ 4271 ]: 0.009020390653319912
loss[ 4272 ]: 0.021167231345211292
loss[ 4273 ]: 0.02238488483250627
loss[ 4274 ]: 0.022785248602863826
loss[ 4275 ]: 0.023076807476485005
loss[ 4276 ]: 0.024531235194239925
loss[ 4277 ]: 0.025462591816114682
loss[ 4278 ]: 0.026423606341558152
loss[ 4279 ]: 0.03773970895150888
loss[ 4280 ]: 0.03872016643662281
loss[ 4281 ]: 0.03413096114122418
loss[ 4282 ]: 0.030346586581134087
loss[ 4283 ]: 0.03820629429882026
loss[ 4284 ]: 0.04225848337966901
loss[ 4285 ]: 0.045930971079242384
loss[ 4286 ]: 0.04775941222008382
loss[ 4287 ]: 0.049353814314206865
loss[ 4288 ]: 0.05138968431345791
loss[ 4289 ]: 0.0480477764628028

loss[ 4500 ]: 0.06992943277196564
loss[ 4501 ]: 0.060774745374609196
loss[ 4502 ]: 0.0499695658418981
loss[ 4503 ]: 0.04270181994527709
loss[ 4504 ]: 0.0417358994887077
loss[ 4505 ]: 0.0441257849522672
loss[ 4506 ]: 0.05177736330397458
loss[ 4507 ]: 0.05168944777501165
loss[ 4508 ]: 0.048481101174063554
loss[ 4509 ]: 0.044352580312875325
loss[ 4510 ]: 0.038891442251268685
loss[ 4511 ]: 0.032674074430643754
loss[ 4512 ]: 0.03701713019130046
loss[ 4513 ]: 0.024990856589789234
loss[ 4514 ]: 0.025260817642825006
loss[ 4515 ]: 0.028838731679997783
loss[ 4516 ]: 0.033462384710126164
loss[ 4517 ]: 0.030161199147097528
loss[ 4518 ]: 0.03010215610210299
loss[ 4519 ]: 0.04456565457717858
loss[ 4520 ]: 0.047231304418427385
loss[ 4521 ]: 0.047058211022800896
loss[ 4522 ]: 0.04485975578131865
loss[ 4523 ]: 0.04251964012441589
loss[ 4524 ]: 0.044704079421996314
loss[ 4525 ]: 0.041636554167443454
loss[ 4526 ]: 0.04633430073223784
loss[ 4527 ]: 0.04571289325928328
loss[ 4528 ]: 0.041599064389047424
lo

loss[ 4744 ]: 0.03720346440378841
loss[ 4745 ]: 0.03431887701298662
loss[ 4746 ]: 0.02869884461485523
loss[ 4747 ]: 0.019809227101573873
loss[ 4748 ]: 0.022445944309202433
loss[ 4749 ]: 0.020691420638322237
loss[ 4750 ]: 0.02074492608124463
loss[ 4751 ]: 0.022580769651427416
loss[ 4752 ]: 0.02754355583740586
loss[ 4753 ]: 0.025847462885949357
loss[ 4754 ]: 0.031355464069482225
loss[ 4755 ]: 0.032789971414143575
loss[ 4756 ]: 0.030895776410572364
loss[ 4757 ]: 0.030726142128531805
loss[ 4758 ]: 0.02892496930978187
loss[ 4759 ]: 0.045403436019069876
loss[ 4760 ]: 0.053676012157399394
loss[ 4761 ]: 0.05256318995999984
loss[ 4762 ]: 0.05900683057276793
loss[ 4763 ]: 0.060410264230735956
loss[ 4764 ]: 0.05860098026340955
loss[ 4765 ]: 0.05437943573072158
loss[ 4766 ]: 0.0635770401655327
loss[ 4767 ]: 0.06482830542415108
loss[ 4768 ]: 0.06049093688780484
loss[ 4769 ]: 0.05170461627425233
loss[ 4770 ]: 0.03711311484804499
loss[ 4771 ]: 0.03636816764198738
loss[ 4772 ]: 0.03502581673484406
los

loss[ 4990 ]: 0.04117423207614479
loss[ 4991 ]: 0.03571732206980745
loss[ 4992 ]: 0.04108892810408846
loss[ 4993 ]: 0.04257016820255351
loss[ 4994 ]: 0.045322119264442455
loss[ 4995 ]: 0.045560312742165834
loss[ 4996 ]: 0.04575839307128357
loss[ 4997 ]: 0.04076027513395145
loss[ 4998 ]: 0.03886851446651029
loss[ 4999 ]: 0.03698946530894115


### Visualize

In [ ]:
fig = plt.figure()
im = plt.imshow(momentum_object.render(), animated=True)
def updatefig(*args):
    position = momentum_object.get_state_action().reshape(1,4)
    predict_position = encoder(standardize_state(position)).detach()
    _predict_position = torch.round(denormalize_state(predict_position).detach())[0]
    momentum_object.step()
    im.set_array(momentum_object.render(_predict_position))
ani = FuncAnimation(fig, updatefig, interval=500, frames=100, blit=True)
plt.show()
updatefig()

In [ ]:
memory.step()
position = memory.states_actions[0].reshape(1,4)
num_next_step = num_step_per_action - memory.current_step
predict_position = encoder(standardize_state(position)).detach()
predict_position = predict_position.repeat((num_next_step, 1))
predict_position_delta_time = torch.cat((predict_position, 
                               torch.arange(num_next_step, device=device).reshape(-1,1)), 
                               axis=1)
torch.round(denormalize_state(decoder(predict_position_delta_time)).detach())